In [1]:
import io
import os
import re
import sys
import ase
import torch
import shutil
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from ase.io import read
from pandas import read_csv
from pandas import read_excel
from ase.visualize import view
import matplotlib.pyplot as plt
from ase.io.eon import write_eon
from sklearn.metrics import r2_score
from ase.io.trajectory import Trajectory
from ase.calculators.lj import LennardJones
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor


In [7]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,0.375,0.400,0.425,0.450,0.475,0.500]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('bsub < run-pyamff.sh')
               
#         print('  ')

In [3]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,0.375,0.400,0.425,0.450,0.475,0.500]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/pred.py .')
#         print('bsub < run-pred.sh')
               
#         print('  ')



In [ ]:
import io
import os
import re
import shutil
import numpy as np
import pandas as pd

def read_rmse(path='/work/mse-minzw/lrz/g1/'):
    df_e = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    df_f = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    eta_list = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
    drs_list = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,0.375,0.400,0.425,0.450,0.475,0.500]
    ii = 0
    for i in eta_list:
        jj = 0
        for j in drs_list:
            path_pl = path + '{}-{:0.3f}'.format(i, j) + '/pyamff.log'
            file = open(path_pl)
            out = file.readlines()
            rmse = out[-1]
            num_list = rmse.split()
            EnergyRMSE = float(num_list[-2])
            ForceRMSE = float(num_list[-1])
            df_e.iloc[ii, jj] = EnergyRMSE
            df_f.iloc[ii, jj] = ForceRMSE
            jj += 1
            print('t',ii,jj)
        ii += 1
        
    with pd.ExcelWriter('./train_rmse.xlsx') as writer1:
        df_e.to_excel(writer1, sheet_name='t_e')
        df_f.to_excel(writer1, sheet_name='t_f') 
                
    
    p_df_e = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    p_df_f = pd.DataFrame(index=np.arange(50),columns=np.arange(50)) 
    pii = 0
    for i in eta_list:
        pjj = 0
        for j in drs_list:            
            path_prl = path + '{}-{:0.3f}'.format(i, j) + '/pred.log'
            pfile = open(path_prl)
            out = pfile.readlines()
            rmse = out[-65]
            pattern = r'[\d.]+'
            number = re.findall(pattern, rmse)
            EnergyRMSE_str = number[0]
            ForceRMSE_str = number[1]
            EnergyRMSE = float(EnergyRMSE_str)
            ForceRMSE = float(ForceRMSE_str)
            p_df_e.iloc[pii, pjj] = EnergyRMSE
            p_df_f.iloc[pii, pjj] = ForceRMSE 
            pjj += 1
            print('p',pii,pjj)
        pii += 1    
        
    with pd.ExcelWriter('./pred_rmse.xlsx') as writer2:
        p_df_e.to_excel(writer2, sheet_name='p_e')
        p_df_f.to_excel(writer2, sheet_name='p_f')        
        
read_rmse()

In [63]:
def read_pyamff_pre_rmse(path='/work/mse-lirz/1/g1/'):
    ii = 0
    df_e = pd.DataFrame(index=np.arange(20),columns=np.arange(20))
    df_f = pd.DataFrame(index=np.arange(20),columns=np.arange(20))
    eta_list = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300]
    for i in eta_list:
        for j in range(18):
            d_ = np.round(0.075 + 0.025 * j, 4)            
            path_prl = path + '{}-{}'.format(i, '%.3f' % d_) + '/pred.log'
            file = open(path_prl)
            out = file.readlines()
            rmse = out[-65]
            pattern = r'[\d.]+'
            number = re.findall(pattern, rmse)
            EnergyRMSE_str = number[0]
            ForceRMSE_str = number[1]
            EnergyRMSE = float(EnergyRMSE_str)
            ForceRMSE = float(ForceRMSE_str)
            df_e.iloc[ii, j] = EnergyRMSE
            df_f.iloc[ii, j] = ForceRMSE 
        ii += 1

    with pd.ExcelWriter('/work/mse-lirz/1/g1/pre_rmse.xlsx') as writer:
        df_e.to_excel(writer, sheet_name='pre_e')
        df_f.to_excel(writer, sheet_name='pre_f')

read_pyamff_pre_rmse()


In [79]:
file = open('/Users/renzhe/Desktop/pred.log')
out = file.readlines()
rmse = out[-65]
pattern = r'[\d.]+'
number = re.findall(pattern, rmse)
EnergyRMSE_str = number[0]
ForceRMSE_str = number[1]
EnergyRMSE = float(EnergyRMSE_str)
ForceRMSE = float(ForceRMSE_str)

In [72]:
out[-65]

'EnergyRMSE=0.011665719779360,ForceRMSE=0.094664441126345\n'

In [80]:
EnergyRMSE

0.01166571977936

In [77]:
file = open('/Users/renzhe/Desktop/pred.log')
out = file.readlines()
rmse = out[-65]
pattern = r'[\d.]+'
number = re.findall(pattern, rmse)
energy_rmse = number[0]
force_rmse = number[1]

In [78]:
energy_rmse

'0.011665719779360'

In [91]:
#d-Rs可能不够，现在从0.525开始加


eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.525,0.550,0.575,0.600,0.625,0.650,0.675,0.700]

for drs_ in drs:
    for eta_ in eta:
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
        print('cp /work/mse-minzw/lrz/g1/data_all/pred.py .')
        print('cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .')
        print('bsub < run-pred.sh')
        
               
        print('  ')

cd /work/mse-minzw/lrz/g1/10-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/100-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/200-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/300-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/400-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/500-0.525
cp /work/mse-minzw/lrz/g1/data_all/pred.py .
cp /work/

In [4]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.725,0.75]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/{}-{:0.3f}.dat .'.format(eta_,drs_))
#         print('mv {}-{:0.3f}.dat fpParas.dat'.format(eta_,drs_))
#         print('bsub < run-pyamff.sh')
        
               
#         print('  ')
    



In [5]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.725,0.75]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pred.sh .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/pred.py .')

#         print('bsub < run-pred.sh')
#         print('  ')

In [98]:
h2o = Trajectory('/Users/renzhe/Desktop/BoLin.2dwater.traj')
len(h2o)


15752

In [99]:
view(h2o)

<Popen: returncode: None args: ['/Users/renzhe/anaconda3/bin/python', '-m', ...>

In [101]:
7506 - 1264

6242

In [102]:
len(eta)

15

In [108]:
eta_list = [10,50,100,200,300,400,500,600,
            700,800,900,1000,1100,1200,1300]
drs_list = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,
            0.250,0.275,0.300,0.325,0.350,0.375,0.400,
            0.425,0.450,0.475,0.500,0.525,0.550,0.575,
            0.600,0.625,0.650,0.675,0.700,0.725,0.75]
print('Number of eta:',len(eta_list)) 
print('Number of drs:',len(drs_list))
print('Grid-Search points are:',len(eta_list)*len(drs_list))

Number of eta: 15
Number of drs: 28
Grid-Search points are: 420


In [104]:
len(drs_list)

28

7506

In [2]:
#计算ef的总loss

In [23]:
te = pd.read_excel('/Users/renzhe/Desktop/g1/g1网格搜索.xlsx',header=None,sheet_name=0)
tf = pd.read_excel('/Users/renzhe/Desktop/g1/g1网格搜索.xlsx',header=None,sheet_name=1)
pe = pd.read_excel('/Users/renzhe/Desktop/g1/g1网格搜索.xlsx',header=None,sheet_name=2)
pf = pd.read_excel('/Users/renzhe/Desktop/g1/g1网格搜索.xlsx',header=None,sheet_name=3)

In [24]:
te,tf,pe,pf

(        0      1         2
 0      10  0.075  0.010779
 1      10  0.100  0.010868
 2      10  0.125  0.011431
 3      10  0.150  0.011018
 4      10  0.175  0.011074
 ..    ...    ...       ...
 415  1100  0.750  0.015398
 416  1200  0.725  0.015278
 417  1200  0.750  0.018921
 418  1300  0.725  0.016310
 419  1300  0.750  0.017574
 
 [420 rows x 3 columns],
         0      1         2
 0      10  0.075  0.190757
 1      10  0.100  0.184966
 2      10  0.125  0.196534
 3      10  0.150  0.197355
 4      10  0.175  0.193684
 ..    ...    ...       ...
 415  1100  0.750  0.309583
 416  1200  0.725  0.289555
 417  1200  0.750  0.345003
 418  1300  0.725  0.313820
 419  1300  0.750  0.334855
 
 [420 rows x 3 columns],
         0      1         2
 0      10  0.075  0.012660
 1      10  0.100  0.013025
 2      10  0.125  0.014431
 3      10  0.150  0.013020
 4      10  0.175  0.012839
 ..    ...    ...       ...
 415  1100  0.750  0.022940
 416  1200  0.725  0.022539
 417  1200  0.750  0.0

In [29]:
t_loss = te.iloc[:,2] + tf.iloc[:,2] * 0.05
p_loss = pe.iloc[:,2] + pf.iloc[:,2] * 0.05

t_loss = pd.DataFrame([t_loss,p_loss])
p_loss = pd.DataFrame(p_loss)

In [35]:
dloss = t_loss - p_loss
dloss

,2
0,0.001513
1,0.000695
2,-0.000831
3,0.000824
4,-0.000140
...,...
415,-0.017939
416,-0.026414
417,-0.014830
418,-0.022911


In [38]:
np.max(np.abs(dloss)),np.min(np.abs(dloss))

(0.166587915916113, 3.167334282664497e-05)

In [39]:
7506-1264

6242

19802

In [43]:
drs_list = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,0.375,0.400,0.425,0.450,0.475,0.500]

In [6]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
       0.375,0.400,0.425,0.450,0.475,0.500,0.725,0.75,0.525,0.550,0.575,0.600,
       0.625,0.650,0.675,0.700]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))

#         print('bsub < run-pred.sh')
#         print('  ')
        
        

In [133]:
fff = data_all[5].get_forces()
fff[1]


array([-0.82328 ,  0.203808,  0.198774])

In [134]:
len_all = np.linalg.norm(fff,axis=1)
len_all

array([1.71545253, 0.87111352, 2.22606827, 1.92956505, 1.05744441,
       0.2798501 , 1.14143655, 1.3029357 , 1.22333086, 0.94737544,
       0.87235873, 1.14959706, 1.0399376 , 0.98709696, 0.94405772,
       1.48463068, 1.16603799, 0.66129999, 2.16048561, 1.19706125,
       1.38509233, 2.02777634, 1.20581742, 1.76556915, 1.11385426,
       0.7369823 , 2.14081655, 0.94441758, 2.60404164, 0.92056859,
       1.18080527, 2.2216669 , 0.8378327 , 0.94659369, 1.02129553,
       1.85894649, 1.33269325, 1.44304057, 2.94758017, 1.70109248,
       1.92395513, 0.47986921, 1.06619219, 0.36258647, 0.89494888,
       1.31924196, 1.73860542, 1.10762489, 1.2903004 , 1.49590452,
       1.31583456, 2.13849296, 0.13638749, 0.77053231, 1.11949273,
       0.71349511, 1.68617809, 1.31252327, 1.92145972, 1.11010327,
       1.4522165 , 2.12840114, 0.623214  , 0.42205349, 1.82867112,
       2.39843726, 1.14717372, 1.15682025, 1.70813175, 2.29322486,
       1.69715624, 1.51172768, 1.10289609, 1.54466834, 0.48125

In [169]:
import io
import os
import re
import ase
import time
import numpy as np
import pandas as pd
from ase.io.trajectory import Trajectory

print("Program Initiated.")
code_time_s = time.perf_counter()
# 统计.traj中所有img原子个数.
def get_all_atoms(img_path):
    num_a = 0
    data_all = Trajectory(img_path)
    for i in range(len(data_all)):
        num_a += len(data_all[i])
    print(num_a)
    return num_a


# 获取一个img的所有atom的受力的合力.
def atom_i_fall(img_i):
    atom_num = len(img_i)
    f_m = img_i.get_forces()
    f_all = np.linalg.norm(f_m, axis=1)
    return f_all


# 获取.traj中所有img的所有atom的受力的合力.
def traj_fall(img_p):
    index_ = []
    f0 = np.array([])
    img_all = Trajectory(img_p)

    # 获取这批.traj中所有img的合力.
    for img in img_all:
        f_ = atom_i_fall(img_i=img)
        _f_all = np.concatenate((f0, f_), axis=0)
        f0 = _f_all

    # 详细记录每个atom对应的每个image.
    for i in range(len(img_all)):
        img_i = img_all[i]
        for j in range(len(img_i)):
            atom_index = '{}   |   {}'.format(i, j)
            index_.append(atom_index)
    data_df = pd.DataFrame()
    data_df['Img   |   Atom'] = pd.DataFrame(index_)
    data_df['Force'] = pd.DataFrame(_f_all)
    data_df.to_csv('./atom_all_fall.csv', encoding='utf-8', index=None)
    return None


traj_fall(img_p='/Users/renzhe/Desktop/g1/1225-6281/train.traj')
code_time_e = time.perf_counter()
print(
    'Program execution completed successfully.Code-Time is {} Sec.'.format(np.round(code_time_e-code_time_s, 2)))

In [161]:
q = np.array([12.07592313, 15.45275999, 11.49983504, 10.06020468, 10.06020468])
w = np.array([-12.07592313, -15.45275999, -11.49983504, -10.06020468, 10.06020468])
e = np.array([0, 1, 2, 3, 10.06020468])
r = np.array([0, -1, -2, -3,])


allll = np.concatenate((q,w,e,r))
allll = pd.DataFrame(allll)
allll.to_csv('/Users/renzhe/Desktop/tessss.xlsx', encoding='utf-8', index=None)

In [107]:
f_all = []
data_all = Trajectory('./1225-6281/train.traj')
for i in range(len(data_all)):
    img = data_all[i]
    img_f = img.get_forces()
    img_f_ = pd.DataFrame(img_f)
    f_x = img_f_.iloc[i,0]
    f_y = img_f_.iloc[i,1]
    f_z = img_f_.iloc[i,2]
    print(i,img_f_.shape)



0 (216, 3)
1 (64, 3)
2 (96, 3)
3 (32, 3)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [108]:
img0 = data_all[4]
f0 = img0.get_forces()
f0

array([[ 9.953257, -2.754965, -6.258655],
       [ 3.219443, 14.694455, -3.534964],
       [-8.6011  , -3.498329,  6.784466],
       [-4.571599, -8.441161,  3.009153]])

In [141]:
a = 3.219443
b = 14.694455
c = 3.534964
(a ** 2 + b ** 2 + c ** 2) ** 0.5

15.452759994854318

In [83]:
fx = f0.iloc[:,0]
fx

0     -0.098072
1      0.033045
2      0.262117
3     -0.053309
4     -0.400366
         ...   
211    0.018473
212    0.236923
213   -0.091114
214   -0.386185
215    0.103523
Name: 0, Length: 216, dtype: float64

In [87]:
np.square(2)

4

In [170]:
import os
import ase
import csv
import time
import torch
import pickle
import numpy as np
import pandas as pd
from ase.io.trajectory import Trajectory

print("Program Initiated. fps --> csv")
code_time_s = time.perf_counter()

# Get the elements of a structure.
def get_symbols(image_path):
    s = []
    img = Trajectory(image_path)
    symbols_all = img[0].get_chemical_symbols()
    symbols_all_set = list(set(symbols_all))
    symbols_all_set.sort()
    if len(symbols_all_set) == 1:
        s.append(symbols_all_set[0])
    elif len(symbols_all_set) == 2:
        s.append(symbols_all_set[0])
        s.append(symbols_all_set[1])
    return s


# xxx.pckl --> xxx.csv
def fps_to_csv(fps_path, img_path, symbol, fps_csvpath):
    img_all = Trajectory(img_path)

    # fprange.pckl --> .csv
    print('Start converting | fprange.pckl --> .csv')
    os.makedirs(fps_csvpath)
    fprange_path = os.path.join(fps_path, 'fprange.pckl')
    with open(fprange_path, 'rb') as f:
        fp_all = pickle.load(f)
        for smb in symbol:
            fp_ = fp_all[smb]
            min_ = fp_[0]
            max_min_ = fp_[2]
            min_np = min_.numpy()
            max_min_np = max_min_.numpy()
            npi = np.concatenate((min_np, max_min_np))
            npi_ = npi.reshape(2, -1)
            dfi = pd.DataFrame(npi_)
            dfi.to_csv(fps_csvpath + '/fpsrange_{}.csv'.format(smb), index=False)

    # fps_i.pckl --> .csv 
    print('Start converting | fps_i.pckl --> .csv')
    data_num = len(img_all)
    for smb in symbol:
        fprange_path1 = os.path.join(fps_csvpath, 'fpsrange_{}.csv'.format(smb))
        fprange_i = pd.read_csv(fprange_path1)
        min_i_np = fprange_i.iloc[0, :]
        min_i = torch.tensor(min_i_np)
        max_min_i_np = fprange_i.iloc[1, :]
        max_min_i = torch.tensor(max_min_i_np)

        for num in range(data_num):
            fp_path = os.path.join(fps_path, 'fps_{}.pckl'.format(num))
            with open(fp_path, 'rb') as f:
                fp_all = pickle.load(f)
                fp_i = fp_all.allElement_fps[smb]
                
                # (no_normalize --> 1)
                fp_i_ori = fp_i  
                fp_i_np_ori = fp_i_ori.numpy()
                dfi_ori = pd.DataFrame(fp_i_np_ori)
                pathfpsi_ori = os.path.join(fps_csvpath, '{}_fps_{}_ori.csv'.format(smb, num))
                dfi.to_csv(pathfpsi_ori, index=False)
                
                # (normalize --> 1)
                fp_i = (fp_i - min_i) / max_min_i 
                fp_i_np = fp_i.numpy()
                dfi = pd.DataFrame(fp_i_np)
                pathfpsi = os.path.join(fps_csvpath, '{}_fps_{}_nor.csv'.format(smb, num))
                dfi.to_csv(pathfpsi, index=False)
                
            if num % 100 == 0:
                print('Completed {}-{}'.format(smb, num))



traj_symbol = get_symbols(image_path='/work/mse-minzw/lrz/g1/data_all/train.traj')
fps_to_csv(fps_path='./fingerprints', 
                img_path='/work/mse-minzw/lrz/g1/data_all/train.traj', 
                symbol=traj_symbol, 
                fps_csvpath='./fps_csv')

code_time_e = time.perf_counter()
print(
        'Program execution completed successfully.Code-Time is {} Sec.'.format(np.round(code_time_e-code_time_s, 2)))


Program Initiated. fps --> csv (normalize --> 1)


FileNotFoundError: [Errno 2] No such file or directory: '/work/mse-minzw/lrz/g1/data_all/train.traj'

In [171]:
(0.02138005-4.72E-33)/4.533778645

0.004715724271977553

In [172]:
0.004837866 * 4.533778645 + 4.72E-33

0.021933813558171568

In [175]:
with open('/Users/renzhe/Desktop/fps_363.pckl', 'rb') as f:
    fp_all = pickle.load(f)


In [178]:
fp_all.allElement_fps['Ge']

tensor([[2.7287e-29, 3.3546e-06, 4.9757e-03, 4.1079e-08, 4.7888e-03, 4.2721e-01,
         1.6022e-02, 2.1380e-02],
        [2.7313e-29, 2.5109e-06, 5.2522e-03, 4.3426e-08, 4.9931e-03, 4.2420e-01,
         1.6152e-02, 2.1672e-02],
        [1.3696e-29, 2.6788e-06, 6.6305e-03, 4.2749e-08, 4.3755e-03, 4.1416e-01,
         1.6938e-02, 2.1266e-02],
        [3.0319e-30, 1.5416e-06, 4.7649e-03, 4.5304e-08, 5.4343e-03, 4.3733e-01,
         1.5452e-02, 2.2062e-02],
        [3.7104e-30, 1.3578e-06, 4.4515e-03, 4.0751e-08, 4.7214e-03, 4.2226e-01,
         1.6520e-02, 2.1430e-02],
        [2.7614e-29, 2.8362e-06, 4.4296e-03, 4.2205e-08, 4.8217e-03, 4.3749e-01,
         1.5664e-02, 2.2321e-02],
        [2.6607e-29, 2.9142e-06, 4.2175e-03, 4.4700e-08, 5.0911e-03, 4.3342e-01,
         1.5574e-02, 2.1993e-02],
        [1.2071e-28, 6.0358e-06, 4.2488e-03, 4.0197e-08, 5.0624e-03, 4.4731e-01,
         1.6240e-02, 2.2517e-02],
        [5.0324e-30, 1.5996e-06, 5.8182e-03, 4.3010e-08, 4.7662e-03, 4.1912e-01,

In [189]:
eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
       0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,0.625,0.650,
       0.675,0.700,0.725,0.750]

for drs_ in drs:
    for eta_ in eta:
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
        print('mkdir fps')
        print('cd fps')
        print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
        print('cp /work/mse-minzw/lrz/g1/data_all/get_fpsall.py .')
        print('cp /work/mse-minzw/lrz/g1/data_all/run-csv-fps.sh .')
        print('cp ../fpParas.dat .')
        print('bsub < run-csv-fps.sh')
        print('  ')
        
        

cd /work/mse-minzw/lrz/g1/10-0.075
mkdir fps
cd fps
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp /work/mse-minzw/lrz/g1/data_all/get_fpsall.py .
cp /work/mse-minzw/lrz/g1/data_all/run-csv-fps.sh .
cp ../fpParas.dat .
bsub < run-csv-fps.sh
  
cd /work/mse-minzw/lrz/g1/50-0.075
mkdir fps
cd fps
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp /work/mse-minzw/lrz/g1/data_all/get_fpsall.py .
cp /work/mse-minzw/lrz/g1/data_all/run-csv-fps.sh .
cp ../fpParas.dat .
bsub < run-csv-fps.sh
  
cd /work/mse-minzw/lrz/g1/100-0.075
mkdir fps
cd fps
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp /work/mse-minzw/lrz/g1/data_all/get_fpsall.py .
cp /work/mse-minzw/lrz/g1/data_all/run-csv-fps.sh .
cp ../fpParas.dat .
bsub < run-csv-fps.sh
  
cd /work/mse-minzw/lrz/g1/200-0.075
mkdir fps
cd fps
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp /work/mse-minzw/lrz/g1/data_all/get_fpsall.py .
cp /work/mse-minzw/lrz/g1/data_all/run-csv-fps.sh .
cp ../fpParas.dat .
bsub < run-csv-fps.sh
  
cd /wo

In [182]:
ele = ['O','Ti']
ele.sort()
print(ele)

['O', 'Ti']


In [183]:
#判断文件夹中文件个数
import os
def get_file_n(f_path):
    pfile = os.listdir(f_path)
    f_n = len(pfile)
    print(f_n)
    return None


eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
       0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,0.625,0.650,
       0.675,0.700,0.725,0.75]

for drs_ in drs:
    for eta_ in eta:
        f_p_ = '/work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_)
        get_file_n(f_path=f_p_)
   

In [187]:
pfile = os.listdir('/Users/renzhe/Desktop/test')

In [188]:
pfile

['.DS_Store',
 'train.traj',
 'fps_csv',
 'fingerprints',
 'Ge',
 '.ipynb_checkpoints']

In [191]:
for i in range(0,301,50):
    print(i)

0
50
100
150
200
250
300


In [195]:
angle_degrees = np.rad2deg(3.1415926)
angle_degrees

179.99999692953102

In [1]:
import pandas as pd

In [15]:
fps_ = pd.read_csv('/Users/renzhe/Desktop/g1/fps-csv-100-0.500/Ge_fps_0_nor.csv')

In [16]:
fps_

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.004404,0.075785,0.260969,0.325880,0.151534,0.089110,0.334347,0.614166,0.475029,8.713794e-07,...,0.018979,0.001949,0.001478,0.148699,0.071180,0.014566,0.000003,3.009238e-13,4.596824e-24,1.922087e-38
1,0.003845,0.069792,0.251601,0.326892,0.153811,0.051776,0.229060,0.578339,0.513174,4.893710e-06,...,0.010255,0.003834,0.021886,0.088725,0.060472,0.023728,0.000923,1.663669e-08,6.680412e-17,1.078670e-28
2,0.002433,0.050545,0.210839,0.322724,0.185818,0.113139,0.375050,0.652342,0.507985,2.210084e-06,...,0.018243,0.003206,0.045010,0.061012,0.019602,0.017150,0.001893,8.521413e-08,8.961614e-16,4.070865e-27
3,0.004673,0.078241,0.263935,0.324967,0.150428,0.093659,0.363261,0.657152,0.423000,2.656652e-07,...,0.009107,0.008105,0.035768,0.122556,0.066160,0.020107,0.000013,3.740081e-12,2.018141e-22,3.277870e-36
4,0.004581,0.077939,0.264903,0.325890,0.150344,0.094392,0.328252,0.528298,0.330933,7.157885e-05,...,0.015881,0.005949,0.021128,0.147840,0.050837,0.011852,0.002545,2.459348e-07,5.632012e-15,5.747180e-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.003139,0.060152,0.231239,0.324973,0.172110,0.101197,0.332943,0.590759,0.427701,3.974985e-05,...,0.014137,0.009831,0.042106,0.024543,0.065137,0.029571,0.000012,1.942686e-12,6.235972e-23,6.053650e-37
212,0.005378,0.086549,0.278584,0.324914,0.151012,0.135289,0.391534,0.561181,0.326024,2.468300e-05,...,0.018275,0.004197,0.007134,0.180911,0.060833,0.034468,0.000542,3.406681e-09,4.568719e-18,2.299677e-30
213,0.003059,0.059294,0.229998,0.325197,0.169566,0.077639,0.286213,0.562826,0.469393,4.681236e-05,...,0.012337,0.005780,0.018396,0.085968,0.050177,0.028909,0.000138,3.068155e-10,1.398428e-19,2.228281e-32
214,0.004446,0.076346,0.262118,0.325963,0.150672,0.083597,0.300820,0.531820,0.382333,1.975411e-05,...,0.010275,0.006263,0.029673,0.069514,0.081668,0.019627,0.000006,7.977034e-13,2.025366e-23,1.467838e-37


In [18]:
res = fps_.iloc[:,:-15]
res

,0,1,2,3,4,5,6,7,8
0,0.004404,0.075785,0.260969,0.325880,0.151534,0.089110,0.334347,0.614166,0.475029
1,0.003845,0.069792,0.251601,0.326892,0.153811,0.051776,0.229060,0.578339,0.513174
2,0.002433,0.050545,0.210839,0.322724,0.185818,0.113139,0.375050,0.652342,0.507985
3,0.004673,0.078241,0.263935,0.324967,0.150428,0.093659,0.363261,0.657152,0.423000
4,0.004581,0.077939,0.264903,0.325890,0.150344,0.094392,0.328252,0.528298,0.330933
...,...,...,...,...,...,...,...,...,...
211,0.003139,0.060152,0.231239,0.324973,0.172110,0.101197,0.332943,0.590759,0.427701
212,0.005378,0.086549,0.278584,0.324914,0.151012,0.135289,0.391534,0.561181,0.326024
213,0.003059,0.059294,0.229998,0.325197,0.169566,0.077639,0.286213,0.562826,0.469393
214,0.004446,0.076346,0.262118,0.325963,0.150672,0.083597,0.300820,0.531820,0.382333


In [2]:
#map图优化

In [3]:
import pandas as pd
data_p_f = pd.read_excel('/Users/renzhe/Desktop/g1-1225撒点.xlsx',sheet_name='pf',header=None)

for i in range(len(data_p_f)):
    if data_p_f.iloc[i,2] >= 0.5:
        eta = data_p_f.iloc[i,0] 
        d_rs = data_p_f.iloc[i,1]
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/'.format(eta,d_rs))
        print('mkdir recal1')
        print('cd recal1')
        
        print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
        print('cp ../fpParas.dat .')
        print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
        
        print('cp ../pred.py .')
        print('cp ../run-pred.sh .')
        print('bsub < run-pyamff.sh')
        print('  ')

cd /work/mse-minzw/lrz/g1/10-0.225/
mkdir recal1
cd recal1
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.375/
mkdir recal1
cd recal1
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.400/
mkdir recal1
cd recal1
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.450/
mkdir recal1
cd recal1
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.525/
mkdir recal1

In [1]:
# for i in range(len(data_p_f)):
#     if data_p_f.iloc[i,2] >= 0.5:
#         eta = data_p_f.iloc[i,0] 
#         d_rs = data_p_f.iloc[i,1]
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal1'.format(eta,d_rs))
#         print('bsub < run-pred.sh')
#         print('  ')

In [5]:
#现在写代码自动更新g1-map-csv文件

In [26]:
path_ = []
for i in range(len(data_p_f)):
    if data_p_f.iloc[i,2] >= 0.5:        
        eta = data_p_f.iloc[i,0] 
        d_rs = data_p_f.iloc[i,1]
        path = '{}-{:0.3f}'.format(eta,d_rs)
        path_.append(path)



In [27]:
path_

['10-0.225',
 '10-0.375',
 '10-0.400',
 '10-0.450',
 '10-0.525',
 '10-0.650',
 '10-0.725',
 '500-0.725',
 '500-0.750',
 '600-0.675',
 '600-0.725',
 '600-0.750',
 '700-0.450',
 '700-0.525',
 '700-0.625',
 '700-0.650',
 '700-0.675',
 '700-0.725',
 '800-0.400',
 '800-0.450',
 '800-0.475',
 '800-0.550',
 '800-0.575',
 '800-0.600',
 '800-0.625',
 '800-0.650',
 '800-0.675',
 '800-0.700',
 '800-0.725',
 '800-0.750',
 '900-0.350',
 '900-0.375',
 '900-0.400',
 '900-0.425',
 '900-0.450',
 '900-0.475',
 '900-0.500',
 '900-0.525',
 '900-0.550',
 '900-0.575',
 '900-0.600',
 '900-0.625',
 '900-0.650',
 '900-0.675',
 '900-0.700',
 '900-0.725',
 '900-0.750',
 '1000-0.400',
 '1000-0.425',
 '1000-0.450',
 '1000-0.475',
 '1000-0.500',
 '1000-0.525',
 '1000-0.550',
 '1000-0.575',
 '1000-0.600',
 '1000-0.625',
 '1000-0.650',
 '1000-0.675',
 '1000-0.700',
 '1000-0.725',
 '1000-0.750',
 '1100-0.375',
 '1100-0.400',
 '1100-0.450',
 '1100-0.475',
 '1100-0.500',
 '1100-0.525',
 '1100-0.550',
 '1100-0.575',
 '11

In [12]:
len(path_)

111

In [17]:
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map

,eta,d_Rs,te,tf,pe,pf
0,10,0.075,0.008993,0.102897,0.020969,0.290511
1,10,0.100,0.010868,0.184966,0.012645,0.138548
2,10,0.125,0.009418,0.105502,0.020224,0.399320
3,10,0.150,0.009373,0.106335,0.020113,0.316235
4,10,0.175,0.009700,0.111144,0.017168,0.364914
...,...,...,...,...,...,...
415,1300,0.650,0.013149,0.156841,0.034427,1.921771
416,1300,0.675,0.014482,0.158863,0.032332,1.508270
417,1300,0.700,0.014090,0.160027,0.030574,1.107743
418,1300,0.725,0.014988,0.175072,0.030568,0.910385


In [23]:
pf = g1_map['pf']
pf

0      0.290511
1      0.138548
2      0.399320
3      0.316235
4      0.364914
         ...   
415    1.921771
416    1.508270
417    1.107743
418    0.910385
419    1.168516
Name: pf, Length: 420, dtype: float64

In [ ]:
path_pyamff = '/Users/renzhe/Desktop/pyamff.log'
path_csv = '/Users/renzhe/Desktop/pred.log'

pfile = open(path_csv)

out = pfile.readlines()
rmse = out[-36]
pattern = r'[\d.]+'
number = re.findall(pattern, rmse)






def read_rmse(path='/work/mse-minzw/lrz/g1/'):
    df_e = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    df_f = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    eta_list = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
    drs_list = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
       0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,
       0.625,0.650,0.675,0.700,0.725,0.75]
    ii = 0
    for i in eta_list:
        jj = 0
        for j in drs_list:
            path_pl = path + '{}-{:0.3f}'.format(i, j) + '/pyamff.log'
            file = open(path_pl)
            out = file.readlines()
            rmse = out[-1]
            num_list = rmse.split()
            EnergyRMSE = float(num_list[-2])
            ForceRMSE = float(num_list[-1])
            df_e.iloc[ii, jj] = EnergyRMSE
            df_f.iloc[ii, jj] = ForceRMSE
            jj += 1
            print('t',i,'-',j)
        ii += 1

    with pd.ExcelWriter('./train_rmse1.xlsx') as writer1:
        df_e.to_excel(writer1, sheet_name='t_e')
        df_f.to_excel(writer1, sheet_name='t_f')
        
        
    p_df_e = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    p_df_f = pd.DataFrame(index=np.arange(50),columns=np.arange(50))
    pii = 0
    for i in eta_list:
        pjj = 0
        for j in drs_list:
            path_prl = path + '{}-{:0.3f}'.format(i, j) + '/pred.log'
            pfile = open(path_prl)
            out = pfile.readlines()
            rmse = out[-36]
            pattern = r'[\d.]+'
            number = re.findall(pattern, rmse)
            EnergyRMSE_str = number[0]
            ForceRMSE_str = number[1]
            EnergyRMSE = float(EnergyRMSE_str)
            ForceRMSE = float(ForceRMSE_str)
            p_df_e.iloc[pii, pjj] = EnergyRMSE
            p_df_f.iloc[pii, pjj] = ForceRMSE
            pjj += 1
            print('p',i,'-',j)
        pii += 1

    with pd.ExcelWriter('./pred_rmse1.xlsx') as writer2:
        p_df_e.to_excel(writer2, sheet_name='p_e')
        p_df_f.to_excel(writer2, sheet_name='p_f')

In [20]:
path_pyamff = '/Users/renzhe/Desktop/pyamff.log'
path_csv = '/Users/renzhe/Desktop/pred.log'

pfile = open(path_csv)

out = pfile.readlines()
rmse = out[-36]
pattern = r'[\d.]+'
number = re.findall(pattern, rmse)
EnergyRMSE_str = number[0]
ForceRMSE_str = number[1]
EnergyRMSE = float(EnergyRMSE_str)
ForceRMSE = float(ForceRMSE_str)

In [19]:
number

['0.021313670069179', '0.867050011328598']

In [21]:
EnergyRMSE, ForceRMSE

(0.021313670069179, 0.867050011328598)

In [28]:
def fix_g1_map_csv(g1mc_path,eta_drs_i):
    g1_map = pd.read_excel(g1mc_path)
    g1_map_len = len(g1_map)
    
    pattern = r'[\d.]+'
    eta_drs_str = re.findall(pattern, eta_drs_i)
    eta_str = number[0]
    drs_str = number[1]
    eta_num = float(EnergyRMSE_str)
    drs_num = float(ForceRMSE_str)
    for i in range(len(g1_map_len)):
        if g1_map['eta'] == eta_num and g1_map['d_Rs'] == drs_num
        pf = 
    
    g1_map = pd.read_excel(path_csv)
    g1_map_pf = g1_map['pf']
    
    
fix_g1_map_csv(path_csv='/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx',eta_drs_i=path_[0])

SyntaxError: duplicate argument 'path_csv' in function definition (3632501566.py, line 1)

In [30]:
path_[0]

'10-0.225'

In [32]:
pattern = r'[\d.]+'
number = re.findall(pattern, path_[0])
number

['10', '0.225']

In [ ]:
path_ = []
for i in range(len(data_p_f)):
    if data_p_f.iloc[i,2] >= 0.5:        
        eta = data_p_f.iloc[i,0] 
        d_rs = data_p_f.iloc[i,1]
        path = '{}-{:0.3f}'.format(eta,d_rs)
        path_.append(path)
        
pattern = r'[\d.]+'
eta_drs_str = re.findall(pattern, eta_drs_i)
eta_str = eta_drs_str[0]
drs_str = eta_drs_str[1]
eta_num = float(EnergyRMSE_str)
drs_num = float(ForceRMSE_str)


In [42]:
# path_ = []
# for i in range(len(data_p_f)):
#     if data_p_f.iloc[i,2] >= 0.5:        
#         eta = data_p_f.iloc[i,0] 
#         d_rs = data_p_f.iloc[i,1]
#         path = '{}-{:0.3f}'.format(eta,d_rs)
#         path_.append(path)
path_ = ['10-0.225',
 '10-0.375',
 '10-0.400',
 '10-0.450',
 '10-0.525',
 '10-0.650',
 '10-0.725',
 '500-0.725',
 '500-0.750',
 '600-0.675',
 '600-0.725',
 '600-0.750',
 '700-0.450',
 '700-0.525',
 '700-0.625',
 '700-0.650',
 '700-0.675',
 '700-0.725',
 '800-0.400',
 '800-0.450',
 '800-0.475',
 '800-0.550',
 '800-0.575',
 '800-0.600',
 '800-0.625',
 '800-0.650',
 '800-0.675',
 '800-0.700',
 '800-0.725',
 '800-0.750',
 '900-0.350',
 '900-0.375',
 '900-0.400',
 '900-0.425',
 '900-0.450',
 '900-0.475',
 '900-0.500',
 '900-0.525',
 '900-0.550',
 '900-0.575',
 '900-0.600',
 '900-0.625',
 '900-0.650',
 '900-0.675',
 '900-0.700',
 '900-0.725',
 '900-0.750',
 '1000-0.400',
 '1000-0.425',
 '1000-0.450',
 '1000-0.475',
 '1000-0.500',
 '1000-0.525',
 '1000-0.550',
 '1000-0.575',
 '1000-0.600',
 '1000-0.625',
 '1000-0.650',
 '1000-0.675',
 '1000-0.700',
 '1000-0.725',
 '1000-0.750',
 '1100-0.375',
 '1100-0.400',
 '1100-0.450',
 '1100-0.475',
 '1100-0.500',
 '1100-0.525',
 '1100-0.550',
 '1100-0.575',
 '1100-0.600',
 '1100-0.625',
 '1100-0.650',
 '1100-0.675',
 '1100-0.700',
 '1100-0.725',
 '1100-0.750',
 '1200-0.375',
 '1200-0.400',
 '1200-0.425',
 '1200-0.450',
 '1200-0.475',
 '1200-0.500',
 '1200-0.525',
 '1200-0.550',
 '1200-0.575',
 '1200-0.600',
 '1200-0.625',
 '1200-0.650',
 '1200-0.675',
 '1200-0.700',
 '1200-0.725',
 '1200-0.750',
 '1300-0.300',
 '1300-0.350',
 '1300-0.375',
 '1300-0.400',
 '1300-0.425',
 '1300-0.450',
 '1300-0.475',
 '1300-0.500',
 '1300-0.525',
 '1300-0.550',
 '1300-0.575',
 '1300-0.600',
 '1300-0.625',
 '1300-0.650',
 '1300-0.675',
 '1300-0.700',
 '1300-0.725',
 '1300-0.750']        
for e_r in path_:
    pattern = r'[\d.]+'
    eta_drs_str = re.findall(pattern, e_r)
    eta_str = eta_drs_str[0]
    drs_str = eta_drs_str[1]
    eta_num = int(eta_str)
    drs_num = float(drs_str)
    drs_num = '{:.3f}'.format(drs_num)
    print(eta_num,'-',drs_num)


10 - 0.225
10 - 0.375
10 - 0.400
10 - 0.450
10 - 0.525
10 - 0.650
10 - 0.725
500 - 0.725
500 - 0.750
600 - 0.675
600 - 0.725
600 - 0.750
700 - 0.450
700 - 0.525
700 - 0.625
700 - 0.650
700 - 0.675
700 - 0.725
800 - 0.400
800 - 0.450
800 - 0.475
800 - 0.550
800 - 0.575
800 - 0.600
800 - 0.625
800 - 0.650
800 - 0.675
800 - 0.700
800 - 0.725
800 - 0.750
900 - 0.350
900 - 0.375
900 - 0.400
900 - 0.425
900 - 0.450
900 - 0.475
900 - 0.500
900 - 0.525
900 - 0.550
900 - 0.575
900 - 0.600
900 - 0.625
900 - 0.650
900 - 0.675
900 - 0.700
900 - 0.725
900 - 0.750
1000 - 0.400
1000 - 0.425
1000 - 0.450
1000 - 0.475
1000 - 0.500
1000 - 0.525
1000 - 0.550
1000 - 0.575
1000 - 0.600
1000 - 0.625
1000 - 0.650
1000 - 0.675
1000 - 0.700
1000 - 0.725
1000 - 0.750
1100 - 0.375
1100 - 0.400
1100 - 0.450
1100 - 0.475
1100 - 0.500
1100 - 0.525
1100 - 0.550
1100 - 0.575
1100 - 0.600
1100 - 0.625
1100 - 0.650
1100 - 0.675
1100 - 0.700
1100 - 0.725
1100 - 0.750
1200 - 0.375
1200 - 0.400
1200 - 0.425
1200 - 0.450
1

In [52]:
import io
import os
import re
import shutil
import numpy as np
import pandas as pd

path_ = ['10-0.225','10-0.375','10-0.400','10-0.450','10-0.525','10-0.650','10-0.725',
 '500-0.725', '500-0.750', '600-0.675', '600-0.725', '600-0.750', '700-0.450', '700-0.525', '700-0.625',
 '700-0.650', '700-0.675', '700-0.725', '800-0.400', '800-0.450', '800-0.475', '800-0.550', '800-0.575', '800-0.600', '800-0.625', '800-0.650',
 '800-0.675', '800-0.700', '800-0.725', '800-0.750', '900-0.350','900-0.375', '900-0.400', '900-0.425',
 '900-0.450', '900-0.475', '900-0.500', '900-0.525', '900-0.550', '900-0.575', '900-0.600', '900-0.625', '900-0.650', '900-0.675', '900-0.700', '900-0.725',
 '900-0.750', '1000-0.400', '1000-0.425', '1000-0.450', '1000-0.475', '1000-0.500', '1000-0.525', '1000-0.550', '1000-0.575',
 '1000-0.600', '1000-0.625', '1000-0.650', '1000-0.675', '1000-0.700', '1000-0.725', '1000-0.750',
 '1100-0.375', '1100-0.400', '1100-0.450', '1100-0.475', '1100-0.500', '1100-0.525', '1100-0.550',
 '1100-0.575', '1100-0.600', '1100-0.625', '1100-0.650', '1100-0.675', '1100-0.700', '1100-0.725',
 '1100-0.750', '1200-0.375', '1200-0.400', '1200-0.425', '1200-0.450', '1200-0.475', '1200-0.500',
 '1200-0.525', '1200-0.550', '1200-0.575', '1200-0.600', '1200-0.625', '1200-0.650', '1200-0.675',
 '1200-0.700', '1200-0.725', '1200-0.750', '1300-0.300', '1300-0.350', '1300-0.375', '1300-0.400',
 '1300-0.425', '1300-0.450', '1300-0.475', '1300-0.500', '1300-0.525', '1300-0.550', '1300-0.575',
 '1300-0.600', '1300-0.625', '1300-0.650', '1300-0.675', '1300-0.700', '1300-0.725', '1300-0.750'] 
df_data = pd.DataFrame(index=path_,columns=np.arange(6))
ii = 0
for e_r in path_:
    pattern = r'[\d.]+'
    eta_drs_str = re.findall(pattern, e_r)
    eta_str = eta_drs_str[0]
    drs_str = eta_drs_str[1]
    eta_num = int(eta_str)
    drs_num = float(drs_str)
    drs_num = '{:.3f}'.format(drs_num)
    path_111 = '/work/mse-minzw/lrz/g1/{}-{}/recal1/'.format(eta_num,drs_num)
    path_111_pred = path_111 + 'pred.log'
    path_111_pyamff = path_111 + 'pyamff.log'
       
    file_pred = open(path_111_pred)
    out_pred = file_pred.readlines()
    rmse_pred = out_pred[-36]
    pattern_pred = r'[\d.]+'
    number_pred = re.findall(pattern_pred, rmse_pred)
    EnergyRMSE_str_pred = number_pred[0]
    ForceRMSE_str_pred = number_pred[1]
    EnergyRMSE_pred = float(EnergyRMSE_str_pred)
    ForceRMSE_pred = float(ForceRMSE_str_pred)
    df_data.iloc[ii, 2] = EnergyRMSE_pred
    df_data.iloc[ii, 3] = ForceRMSE_pred
   
    file_pyamff = open(path_111_pyamff)
    out_pyamff = file_pyamff.readlines()
    rmse_pyamff = out_pyamff[-1]
    num_list_pyamff = rmse_pyamff.split()
    EnergyRMSE_pyamff = float(num_list_pyamff[-2])
    ForceRMSE_pyamff = float(num_list_pyamff[-1])
    df_data.iloc[ii, 0] = EnergyRMSE_pyamff
    df_data.iloc[ii, 1] = ForceRMSE_pyamff
    
    ii += 1

with pd.ExcelWriter('./g1_map_opt.xlsx') as writer2:
    df_data.to_excel(writer2, sheet_name='g1_opt')
        




/work/mse-minzw/lrz/g1/10-0.225/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.375/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.400/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.450/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.525/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.650/recal1/pyamff.log
/work/mse-minzw/lrz/g1/10-0.725/recal1/pyamff.log
/work/mse-minzw/lrz/g1/500-0.725/recal1/pyamff.log
/work/mse-minzw/lrz/g1/500-0.750/recal1/pyamff.log
/work/mse-minzw/lrz/g1/600-0.675/recal1/pyamff.log
/work/mse-minzw/lrz/g1/600-0.725/recal1/pyamff.log
/work/mse-minzw/lrz/g1/600-0.750/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.450/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.525/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.625/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.650/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.675/recal1/pyamff.log
/work/mse-minzw/lrz/g1/700-0.725/recal1/pyamff.log
/work/mse-minzw/lrz/g1/800-0.400/recal1/pyamff.log
/work/mse-minzw/lrz/g1/800-0.450/recal

In [56]:

path_ = ['10-0.225','10-0.375','10-0.400','10-0.450','10-0.525','10-0.650','10-0.725',
 '500-0.725', '500-0.750', '600-0.675', '600-0.725', '600-0.750', '700-0.450', '700-0.525', '700-0.625',
 '700-0.650', '700-0.675', '700-0.725', '800-0.400', '800-0.450', '800-0.475', '800-0.550', '800-0.575', '800-0.600', '800-0.625', '800-0.650',
 '800-0.675', '800-0.700', '800-0.725', '800-0.750', '900-0.350','900-0.375', '900-0.400', '900-0.425',
 '900-0.450', '900-0.475', '900-0.500', '900-0.525', '900-0.550', '900-0.575', '900-0.600', '900-0.625', '900-0.650', '900-0.675', '900-0.700', '900-0.725',
 '900-0.750', '1000-0.400', '1000-0.425', '1000-0.450', '1000-0.475', '1000-0.500', '1000-0.525', '1000-0.550', '1000-0.575',
 '1000-0.600', '1000-0.625', '1000-0.650', '1000-0.675', '1000-0.700', '1000-0.725', '1000-0.750',
 '1100-0.375', '1100-0.400', '1100-0.450', '1100-0.475', '1100-0.500', '1100-0.525', '1100-0.550',
 '1100-0.575', '1100-0.600', '1100-0.625', '1100-0.650', '1100-0.675', '1100-0.700', '1100-0.725',
 '1100-0.750', '1200-0.375', '1200-0.400', '1200-0.425', '1200-0.450', '1200-0.475', '1200-0.500',
 '1200-0.525', '1200-0.550', '1200-0.575', '1200-0.600', '1200-0.625', '1200-0.650', '1200-0.675',
 '1200-0.700', '1200-0.725', '1200-0.750', '1300-0.300', '1300-0.350', '1300-0.375', '1300-0.400',
 '1300-0.425', '1300-0.450', '1300-0.475', '1300-0.500', '1300-0.525', '1300-0.550', '1300-0.575',
 '1300-0.600', '1300-0.625', '1300-0.650', '1300-0.675', '1300-0.700', '1300-0.725', '1300-0.750'] 
df_data = pd.DataFrame(index=path_,columns=np.arange(4))

In [57]:
df_data

,0,1,2,3
10-0.225,NaN,NaN,NaN,NaN
10-0.375,NaN,NaN,NaN,NaN
10-0.400,NaN,NaN,NaN,NaN
10-0.450,NaN,NaN,NaN,NaN
10-0.525,NaN,NaN,NaN,NaN
...,...,...,...,...
1300-0.650,NaN,NaN,NaN,NaN
1300-0.675,NaN,NaN,NaN,NaN
1300-0.700,NaN,NaN,NaN,NaN
1300-0.725,NaN,NaN,NaN,NaN


In [58]:
#recal2

In [70]:
path_2 = ['10-0.125','10-0.175','10-0.250','10-0.275','10-0.450','10-0.475','10-0.725',
         '50-0.250','50-0.425','50-0.750',
         '200-0.625',
         '300-0.650','300-0.725',
         '400-0.475','400-0.600','400-0.625',
         '500-0.475','500-0.575','500-0.750',
         '600-0.425','600-0.450','600-0.550','600-0.575','600-0.675',
         '700-0.550','700-0.625','700-0.725',
         '800-0.475','800-0.650',
         '900-0.075','900-0.275','900-0.325','900-0.400','900-0.450','900-0.625',
         '1000-0.075','1000-0.400','1000-0.450','1000-0.625','1000-0.725',
         '1100-0.325','1100-0.400','1100-0.625','1100-0.700',
         '1200-0.450','1100-0.550','1100-0.650','1100-0.750',
         '1300-0.400','1300-0.450','1300-0.650','1300-0.675'
        
        ] 

for e_r in path_2:
    path_i = 'cd /work/mse-minzw/lrz/g1/' + e_r +'/recal2'
    print(path_i)
    print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
    print('cp ../fpParas.dat .')
    print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
    print('cp ../pred.py .')
    print('cp ../run-pred.sh .')
    print('bsub < run-pyamff.sh')
    print('  ')


cd /work/mse-minzw/lrz/g1/10-0.125/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.175/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.250/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.275/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.450/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp 

In [73]:
path_2 = ['10-0.125','10-0.175','10-0.250','10-0.275','10-0.450','10-0.475','10-0.725',
         '50-0.250','50-0.425','50-0.750',
         '200-0.625',
         '300-0.650','300-0.725',
         '400-0.475','400-0.600','400-0.625',
         '500-0.475','500-0.575','500-0.750',
         '600-0.425','600-0.450','600-0.550','600-0.575','600-0.675',
         '700-0.550','700-0.625','700-0.725',
         '800-0.475','800-0.650',
         '900-0.075','900-0.275','900-0.325','900-0.400','900-0.450','900-0.625',
         '1000-0.075','1000-0.400','1000-0.450','1000-0.625','1000-0.725',
         '1100-0.325','1100-0.400','1100-0.625','1100-0.700',
         '1200-0.450','1100-0.550','1100-0.650','1100-0.750',
         '1300-0.400','1300-0.450','1300-0.650','1300-0.675'
        
        ] 

for e_r in path_2:
    path_i = 'cd /work/mse-minzw/lrz/g1/' + e_r +'/recal2'
    print(path_i)
    print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
    print('bsub < run-pred.sh')
    print(' ')

cd /work/mse-minzw/lrz/g1/10-0.125/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.175/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.250/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.275/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.450/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.475/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/10-0.725/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/50-0.250/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
bsub < run-pred.sh
 
cd /work/mse-minzw/lrz/g1/50-0.425/recal2
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.

416

156

In [ ]:
import io
import os
import re
import shutil
import numpy as np
import pandas as pd

_path_1 = ['10-0.225', '10-0.375', '10-0.400', '10-0.450', '10-0.525', '10-0.650', '10-0.725',
           '500-0.725', '500-0.750', '600-0.675', '600-0.725', '600-0.750', '700-0.450', '700-0.525', '700-0.625',
           '700-0.650', '700-0.675', '700-0.725', '800-0.400', '800-0.450', '800-0.475', '800-0.550', '800-0.575',
           '800-0.600', '800-0.625', '800-0.650',
           '800-0.675', '800-0.700', '800-0.725', '800-0.750', '900-0.350', '900-0.375', '900-0.400', '900-0.425',
           '900-0.450', '900-0.475', '900-0.500', '900-0.525', '900-0.550', '900-0.575', '900-0.600', '900-0.625',
           '900-0.650', '900-0.675', '900-0.700', '900-0.725',
           '900-0.750', '1000-0.400', '1000-0.425', '1000-0.450', '1000-0.475', '1000-0.500', '1000-0.525',
           '1000-0.550', '1000-0.575',
           '1000-0.600', '1000-0.625', '1000-0.650', '1000-0.675', '1000-0.700', '1000-0.725', '1000-0.750',
           '1100-0.375', '1100-0.400', '1100-0.450', '1100-0.475', '1100-0.500', '1100-0.525', '1100-0.550',
           '1100-0.575', '1100-0.600', '1100-0.625', '1100-0.650', '1100-0.675', '1100-0.700', '1100-0.725',
           '1100-0.750', '1200-0.375', '1200-0.400', '1200-0.425', '1200-0.450', '1200-0.475', '1200-0.500',
           '1200-0.525', '1200-0.550', '1200-0.575', '1200-0.600', '1200-0.625', '1200-0.650', '1200-0.675',
           '1200-0.700', '1200-0.725', '1200-0.750', '1300-0.300', '1300-0.350', '1300-0.375', '1300-0.400',
           '1300-0.425', '1300-0.450', '1300-0.475', '1300-0.500', '1300-0.525', '1300-0.550', '1300-0.575',
           '1300-0.600', '1300-0.625', '1300-0.650', '1300-0.675', '1300-0.700', '1300-0.725', '1300-0.750']

_path_2 = ['10-0.125', '10-0.175', '10-0.250', '10-0.275', '10-0.450', '10-0.475', '10-0.725',
           '50-0.250', '50-0.425', '50-0.750',
           '200-0.625',
           '300-0.650', '300-0.725',
           '400-0.475', '400-0.600', '400-0.625',
           '500-0.475', '500-0.575', '500-0.750',
           '600-0.425', '600-0.450', '600-0.550', '600-0.575', '600-0.675',
           '700-0.550', '700-0.625', '700-0.725',
           '800-0.475', '800-0.650',
           '900-0.075', '900-0.275', '900-0.325', '900-0.400', '900-0.450', '900-0.625',
           '1000-0.075', '1000-0.400', '1000-0.450', '1000-0.625', '1000-0.725',
           '1100-0.325', '1100-0.400', '1100-0.625', '1100-0.700',
           '1200-0.450', '1100-0.550', '1100-0.650', '1100-0.750',
           '1300-0.400', '1300-0.450', '1300-0.650', '1300-0.675']

path_0000 = _path_2

df_data = pd.DataFrame(index=path_0000, columns=np.arange(4))
ii = 0
for e_r in path_0000:
    pattern = r'[\d.]+'
    eta_drs_str = re.findall(pattern, e_r)
    eta_str = eta_drs_str[0]
    drs_str = eta_drs_str[1]
    eta_num = int(eta_str)
    drs_num = float(drs_str)
    drs_num = '{:.3f}'.format(drs_num)
    path_111 = '/work/mse-minzw/lrz/g1/{}-{}/recal2/'.format(eta_num, drs_num)
    path_111_pred = path_111 + 'pred.log'
    path_111_pyamff = path_111 + 'pyamff.log'

    file_pred = open(path_111_pred)
    out_pred = file_pred.readlines()
    rmse_pred = out_pred[-36]
    pattern_pred = r'[\d.]+'
    number_pred = re.findall(pattern_pred, rmse_pred)
    EnergyRMSE_str_pred = number_pred[0]
    ForceRMSE_str_pred = number_pred[1]
    EnergyRMSE_pred = float(EnergyRMSE_str_pred)
    ForceRMSE_pred = float(ForceRMSE_str_pred)
    df_data.iloc[ii, 2] = EnergyRMSE_pred
    df_data.iloc[ii, 3] = ForceRMSE_pred

    file_pyamff = open(path_111_pyamff)
    out_pyamff = file_pyamff.readlines()
    rmse_pyamff = out_pyamff[-1]
    num_list_pyamff = rmse_pyamff.split()
    EnergyRMSE_pyamff = float(num_list_pyamff[-2])
    ForceRMSE_pyamff = float(num_list_pyamff[-1])
    df_data.iloc[ii, 0] = EnergyRMSE_pyamff
    df_data.iloc[ii, 1] = ForceRMSE_pyamff

    ii += 1
    print(ii)

with pd.ExcelWriter('./g1_map_opt.xlsx') as writer2:
    df_data.to_excel(writer2, sheet_name='g1_opt')


In [74]:
#if data[i] > data[i-1] and ata[i] > data[i+1] return i

In [75]:
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map

,eta,d_Rs,te,tf,pe,pf
0,10,0.075,0.008993,0.102897,0.020969,0.290511
1,10,0.100,0.010868,0.184966,0.012645,0.138548
2,10,0.125,0.009445,0.107986,0.017318,0.255143
3,10,0.150,0.009373,0.106335,0.020113,0.316235
4,10,0.175,0.009700,0.111144,0.017168,0.364914
...,...,...,...,...,...,...
415,1300,0.650,0.014170,0.159003,0.034261,1.376748
416,1300,0.675,0.014482,0.161866,0.032332,1.386949
417,1300,0.700,0.014090,0.162596,0.030574,0.898565
418,1300,0.725,0.014988,0.175072,0.030568,0.910385


In [78]:
g1_map_pf = g1_map['pf']
g1_map_pf

0      0.290511
1      0.138548
2      0.255143
3      0.316235
4      0.364914
         ...   
415    1.376748
416    1.386949
417    0.898565
418    0.910385
419    1.168516
Name: pf, Length: 420, dtype: float64

In [2]:
# retrain = []
# for i in range(1, len(g1_map_pf)-1):
#     if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
#         retrain.append(i)
#         eta = data_p_f.iloc[i,0]
#         d_rs = data_p_f.iloc[i,1]
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta,d_rs))
#         print('mkdir recal3')
#         print('cd recal3')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')


In [93]:
len(retrain)

116

In [3]:
# retrain = []
# for i in range(1, len(g1_map_pf)-1):
#     if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
#         eta = data_p_f.iloc[i,0]
#         d_rs = data_p_f.iloc[i,1]
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal3'.format(eta,d_rs))
#         inf_ = '{}-{:0.3f}'.format(eta,d_rs)
#         retrain.append(inf_)

In [101]:
len(retrain)

116

In [ ]:
path = ['10-0.175', '10-0.250', '10-0.300',  '10-0.375',  '10-0.450', '10-0.550', '10-0.650', '10-0.750',
 '50-0.175', '50-0.250', '50-0.300', '50-0.375', '50-0.425', '50-0.500', '50-0.550', '50-0.650', '50-0.750',
 '100-0.125', '100-0.200', '100-0.275', '100-0.350', '100-0.400', '100-0.450', '100-0.550', '100-0.600', '100-0.750',
 '200-0.200', '200-0.250', '200-0.425', '200-0.625', '200-0.725', '300-0.100', '300-0.225', '300-0.275', '300-0.475',
 '300-0.575', '300-0.675', '400-0.125', '400-0.250', '400-0.300', '400-0.550', '400-0.600', '400-0.650', '400-0.750',
 '500-0.100', '500-0.225', '500-0.300', '500-0.400', '500-0.450', '500-0.500', '500-0.575', '500-0.625', '500-0.750',
 '600-0.100', '600-0.175', '600-0.225', '600-0.425', '600-0.550', '600-0.675', '600-0.750', '700-0.150', '700-0.425',
 '700-0.500', '700-0.575', '700-0.625', '700-0.725', '800-0.150', '800-0.200', '800-0.275', '800-0.475', '800-0.600',
 '800-0.675', '800-0.750', '900-0.100', '900-0.150', '900-0.225', '900-0.275', '900-0.400', '900-0.475', '900-0.525',
 '900-0.575', '900-0.650', '900-0.750', '1000-0.100', '1000-0.175', '1000-0.275', '1000-0.325', '1000-0.400', '1000-0.450',
 '1000-0.550', '1000-0.600', '1000-0.675', '1000-0.725', '1100-0.100', '1100-0.175', '1100-0.275', '1100-0.325', '1100-0.400',
 '1100-0.475', '1100-0.525', '1100-0.575', '1100-0.625','1100-0.700', '1200-0.150', '1200-0.400', '1200-0.550',
 '1200-0.650', '1200-0.725', '1300-0.175', '1300-0.275', '1300-0.350', '1300-0.400', '1300-0.450', '1300-0.500', '1300-0.550', '1300-0.675']

In [103]:
#recal4

In [4]:
# g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
# retrain = []
# g1_map_pf = g1_map['pf']
# for i in range(1, len(g1_map_pf)-1):
#     if (g1_map_pf[i] > g1_map_pf[i-1]) and (g1_map_pf[i] > g1_map_pf[i+1]):
#         eta = data_p_f.iloc[i,0]
#         d_rs = data_p_f.iloc[i,1]
#         inf_ = '{}-{:0.3f}'.format(eta,d_rs)
#         retrain.append(inf_)
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta,d_rs))
#         print('mkdir recal4')
#         print('cd recal4')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

In [5]:
# g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
# retrain = []
# g1_map_pf = g1_map['pf']
# for i in range(1, len(g1_map_pf)-1):
#     if (g1_map_pf[i] > g1_map_pf[i-1]) and (g1_map_pf[i] > g1_map_pf[i+1]):
#         eta = data_p_f.iloc[i,0]
#         d_rs = data_p_f.iloc[i,1]
#         inf_ = '{}-{:0.3f}'.format(eta,d_rs)
#         retrain.append(inf_)
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal4'.format(eta,d_rs))
#         print('bsub < run-pred.sh')
#         print('  ')

In [ ]:
#recal5

In [6]:
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('mkdir recal5')
#         print('cd recal5')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')


In [7]:
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300]
# _path_5 = []

# for drs_ in drs:
#     for eta_ in eta:
#         path = '{}-{:0.3f}'.format(eta_,drs_)
#         _path_5.append(path)

# _path_5

In [8]:
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal5'.format(eta_,drs_))
#         print('bsub < run-pred.sh')
#         print('  ')


In [ ]:
#recal6

In [9]:
# eta = [10,50,100,200,300,400]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
#        0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,0.625,0.650,
#        0.675,0.700,0.725,0.75]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('mkdir recal6')
#         print('cd recal6')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

In [10]:
# eta = [10,50,100,200,300,400]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
#        0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,0.625,0.650,
#        0.675,0.700,0.725,0.75]
# _path_6 = []

# for eta_ in eta:
#     for drs_ in drs:
#         path = '{}-{:0.3f}'.format(eta_,drs_)
#         _path_6.append(path)

# _path_6

In [11]:
# eta = [10,50,100,200,300,400]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200,0.225,0.250,0.275,0.300,0.325,0.350,
#        0.375,0.400,0.425,0.450,0.475,0.500,0.525,0.550,0.575,0.600,0.625,0.650,
#        0.675,0.700,0.725,0.75]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal6'.format(eta_,drs_))
#         print('bsub < run-pred.sh')
#         print('  ')

In [ ]:
#recal8

In [40]:
retrain = []
import numpy as np
import pandas as pd
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map_pf = g1_map['pf']
num_ = 0
for i in range(1, len(g1_map_pf)-1):
    if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
        retrain.append(i)
        eta = g1_map.iloc[i,0]
        d_rs = g1_map.iloc[i,1]
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta,d_rs))
        print('mkdir recal8')
        print('cd recal8')
        print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
        print('cp ../fpParas.dat .')
        print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
        print('cp ../pred.py .')
        print('cp ../run-pred.sh .')
        print('bsub < run-pyamff.sh')
        print('  ')
        num_ += 1


cd /work/mse-minzw/lrz/g1/10-0.150
mkdir recal8
cd recal8
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.225
mkdir recal8
cd recal8
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.275
mkdir recal8
cd recal8
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.325
mkdir recal8
cd recal8
cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .
cp ../fpParas.dat .
cp /work/mse-minzw/lrz/g1/data_all/config.ini .
cp ../pred.py .
cp ../run-pred.sh .
bsub < run-pyamff.sh
  
cd /work/mse-minzw/lrz/g1/10-0.375
mkdir recal8
cd r

In [41]:
num_

100

In [42]:
retrain = []
import numpy as np
import pandas as pd
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map_pf = g1_map['pf']
num_ = 0
for i in range(1, len(g1_map_pf)-1):
    if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
        retrain.append(i)
        eta = g1_map.iloc[i,0]
        d_rs = g1_map.iloc[i,1]
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal8'.format(eta,d_rs))
        print('bsub < run-pred.sh')
        print('  ')


cd /work/mse-minzw/lrz/g1/10-0.150/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.225/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.275/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.325/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.375/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.450/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.575/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.625/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.675/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/10-0.725/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.200/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.250/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.325/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.400/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.575/recal8
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/50-0.725/recal

In [45]:
retrain = []
import numpy as np
import pandas as pd
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map_pf = g1_map['pf']
num_ = 0
for i in range(1, len(g1_map_pf)-1):
    if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
        retrain.append(i)
        eta = g1_map.iloc[i,0]
        d_rs = g1_map.iloc[i,1]
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal1'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal2'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal3'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal4'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal5'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal6'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal7'.format(eta,d_rs))
        print('rm 6* 5*')
        print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal8'.format(eta,d_rs))
        print('rm 6* 5*')
        print('  ')


cd /work/mse-minzw/lrz/g1/10-0.150/
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal1
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal2
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal3
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal4
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal5
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal6
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal7
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.150/recal8
rm 6* 5*
  
cd /work/mse-minzw/lrz/g1/10-0.225/
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal1
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal2
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal3
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal4
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal5
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal6
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal7
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.225/recal8
rm 6* 5*
  
cd /work/mse-minzw/lrz/g1/10-0.275/
rm 6* 5*
cd /work/mse-minzw/lrz/g1/10-0.275/recal1
r

In [64]:
retrain = []
import numpy as np
import pandas as pd
g1_map = pd.read_excel('/Users/renzhe/Desktop/g1/g1_map_data1210.xlsx')
g1_map_pf = g1_map['pf']
num_ = 0
for i in range(1, len(g1_map_pf)-1):
    if g1_map_pf[i] > g1_map_pf[i-1] and g1_map_pf[i] > g1_map_pf[i+1]:
        eta = g1_map.iloc[i,0]
        d_rs = g1_map.iloc[i,1]
        path__ = '{}-{:0.3f}'.format(eta,d_rs)
        retrain.append(path__)

retrain

['10-0.150',
 '10-0.225',
 '10-0.275',
 '10-0.325',
 '10-0.375',
 '10-0.450',
 '10-0.575',
 '10-0.625',
 '10-0.675',
 '10-0.725',
 '50-0.200',
 '50-0.250',
 '50-0.325',
 '50-0.400',
 '50-0.575',
 '50-0.725',
 '100-0.125',
 '100-0.200',
 '100-0.250',
 '100-0.350',
 '100-0.450',
 '100-0.600',
 '100-0.750',
 '200-0.100',
 '200-0.200',
 '200-0.300',
 '200-0.375',
 '200-0.450',
 '200-0.550',
 '200-0.600',
 '200-0.750',
 '300-0.150',
 '300-0.225',
 '300-0.300',
 '300-0.400',
 '300-0.450',
 '300-0.500',
 '300-0.575',
 '300-0.700',
 '400-0.125',
 '400-0.225',
 '400-0.275',
 '400-0.575',
 '400-0.700',
 '500-0.125',
 '500-0.250',
 '500-0.425',
 '500-0.575',
 '500-0.650',
 '500-0.725',
 '600-0.200',
 '600-0.400',
 '600-0.575',
 '600-0.650',
 '600-0.700',
 '600-0.750',
 '700-0.150',
 '700-0.425',
 '700-0.550',
 '700-0.625',
 '700-0.750',
 '800-0.125',
 '800-0.250',
 '800-0.450',
 '800-0.500',
 '800-0.675',
 '800-0.750',
 '900-0.200',
 '900-0.275',
 '900-0.400',
 '900-0.500',
 '900-0.575',
 '900-0.

In [12]:
# #recal9
# eta = [600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('mkdir recal9')
#         print('cd recal9')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

In [13]:
# #recal9
# eta = [600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100,0.125,0.150,0.175,0.200]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal9'.format(eta_,drs_))
#         print('bsub < run-pred.sh')
#         print('  ')

In [14]:
# retrain = []
# eta_ = [600,700,800,900,1000,1100,1200,1300]
# drs_ = [0.075,0.100,0.125,0.150,0.175,0.200]
# import numpy as np
# import pandas as pd


# for eta in eta_:
#     for d_rs in drs_:
#         path__ = '{}-{:0.3f}'.format(eta,d_rs)
#         retrain.append(path__)
# retrain

In [15]:
# #recal10
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('mkdir recal10')
#         print('cd recal10')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')
        
# #recal11
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}'.format(eta_,drs_))
#         print('mkdir recal11')
#         print('cd recal11')
#         print('cp /work/mse-minzw/lrz/g1/data_all/run-pyamff.sh .')
#         print('cp ../fpParas.dat .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/config.ini .')
#         print('cp ../pred.py .')
#         print('cp ../run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

In [16]:
# #pred-recal10 & recal11
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100]

# for drs_ in drs:
#     for eta_ in eta:
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal10'.format(eta_,drs_))
#         print('bsub < run-pred.sh')
#         print('  ')
#         print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal11'.format(eta_,drs_))
#         print('bsub < run-pred.sh')
#         print('  ')


In [17]:
# retrain10 = []
# eta_ = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs_ = [0.075,0.100]
# import numpy as np
# import pandas as pd

# for eta in eta_:
#     for d_rs in drs_:
#         path__ = '{}-{:0.3f}'.format(eta,d_rs)
#         retrain10.append(path__)
# retrain10

In [18]:
# #pred-recal10 & recal11
# eta = [10,50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300]
# drs = [0.075,0.100]
# recal_num = [1,2,3,4,5,6,7,8,9,10,11]
# for drs_ in drs:
#     for eta_ in eta:
#         for re in recal_num:
#             print('cd /work/mse-minzw/lrz/g1/{}-{:0.3f}/recal{}'.format(eta_,drs_,re))
#             print('rm 5* 6*')
#             print('  ')

In [19]:
# qq = [1049 , 1464 , 2083 , 3013 , 392 , 556 , 722 ,1264 , 1764 , 2529  ,3557  ,432 , 639 , 877]
# rec = [1,2,3,4,5]
# for q in qq:
#     for recc in rec:
#         print('cd /work/mse-minzw/lrz/aks/clustering/{}/recal{}'.format(q,recc))
#         print('rm 5* 6*')
#         print('  ')

In [20]:
# qq = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
# rec = [1,2,3,4]
# for q in qq:
#     print('cd /work/mse-minzw/lrz/aks/pdh/{}/cur'.format(q,recc))
#     print('rm 5* 6*')
#     print('  ')